In [ ]:
# Remote Training on SSH with CUDA - Hydra Sweep

This notebook lets you launch a **Hydra multirun sweep** on an SSH machine with CUDA and safely disconnect while training continues.

**Sweep Configuration:**
- `model.hidden_channels`: [32, 64, 128, 256, 512]
- `model.conv_types`: 1-layer (GEN, GCN, GAT) and 2-layer combinations

**How it works:**
- Cell 2: Launches sweep as a background process with `nohup` (survives SSH disconnect)
- Cell 3: Monitor recent sweep logs
- Cell 4: Check GPU utilization
- Cell 5: Stop sweep if needed
- Cell 6: View sweep results summary

**Workflow:**
1. Run Cell 2 to start sweep
2. Disconnect from SSH/close notebook - sweep continues!
3. Reconnect later and run Cell 3 to check progress
4. Check results in `multirun/` directory when done

In [ ]:
import os
from datetime import datetime

# Create logs directory if it doesn't exist
os.makedirs('logs', exist_ok=True)

# Generate unique log filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_file = f"logs/sweep_{timestamp}.log"

print(f"Starting Hydra sweep in background...")
print(f"Logs will be written to: {log_file}")
print(f"\nSweep parameters:")
print(f"  hidden_channels: 32, 64, 128, 256, 512")
print(f"  conv_types: GEN, GCN, GAT (1-layer)")
print(f"  conv_types: GEN,GEN / GCN,GCN / GAT,GAT (2-layer)")
print(f"  Total runs: 5 × 6 = 30 experiments")

# Hydra multirun command with sweep parameters
# -m flag enables multirun mode
# Sweep over hidden_channels and conv_types
cmd = (
    f"nohup python main.py -m "
    f"'model.hidden_channels=choice(32,64,128,256,512)' "
    f"'model.conv_types=choice(GEN,GCN,GAT,\"GEN,GEN\",\"GCN,GCN\",\"GAT,GAT\")' "
    f"&> {log_file} &"
)

print(f"\nCommand: {cmd}")
os.system(cmd)

print("\n✓ Hydra sweep launched in background!")
print(f"\nUseful commands:")
print(f"  Monitor live:     tail -f {log_file}")
print(f"  Check status:     ps aux | grep 'python main.py'")
print(f"  Kill if needed:   pkill -f 'python main.py'")
print(f"  GPU usage:        nvidia-smi")
print(f"  Results:          ls -lh multirun/")
print(f"\nResults will be saved in: multirun/YYYY-MM-DD/HH-MM-SS/")

Starting training in background...
Logs will be written to: logs/training_20251027_194150.log
To monitor progress: tail -f logs/training_20251027_194150.log
To check if still running: ps aux | grep 'python main.py'

✓ Training launched in background!

Useful commands:
  Monitor live:     tail -f logs/training_20251027_194150.log
  Check status:     ps aux | grep 'python main.py'
  Kill if needed:   pkill -f 'python main.py'
  GPU usage:        nvidia-smi
[2025-10-27 19:41:53,146][__main__][INFO] - ==================================================
[2025-10-27 19:41:53,147][__main__][INFO] - CONFIGURATION
[2025-10-27 19:41:53,147][__main__][INFO] - ==================================================
[2025-10-27 19:41:53,150][__main__][INFO] - 
dataset:
  num_graphs: 5000
  num_steps: 6
  dt: 0.01
  train_ratio: 0.8
  batch_size: 4
  shuffle: true
  drop_last: false
  wave_speed: 1.0
  damping: 1.0
  num_nodes: 100
  domain_length: 1.0
  force:
    margin: 0.1
    sign: -1.0
    location:

In [ ]:
# Monitor the most recent sweep log
import glob
import time

# Find most recent log file
log_files = sorted(glob.glob('logs/sweep_*.log'))
if log_files:
    latest_log = log_files[-1]
    print(f"Monitoring: {latest_log}\n")
    print("="*70)
    
    # Display last 50 lines to see multiple runs
    with open(latest_log, 'r') as f:
        lines = f.readlines()
        for line in lines[-50:]:
            print(line.rstrip())
    
    print("\n" + "="*70)
    print(f"Total lines in log: {len(lines)}")
else:
    print("No sweep logs found yet.")

Monitoring: logs/training_20251027_180057.log



In [6]:
# Check GPU utilization
import subprocess

result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
print(result.stdout)

Mon Oct 27 19:41:42 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.230.02             Driver Version: 535.230.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:01:00.0 Off |                  N/A |
| 30%   51C    P2             123W / 370W |    372MiB / 24576MiB |     15%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
# Stop training if needed (kills all python main.py processes)
import subprocess

print("Stopping all training processes...")
subprocess.run(['pkill', '-f', 'python main.py'])
print("✓ Training processes stopped")

# Verify
result = subprocess.run(['pgrep', '-f', 'python main.py'], capture_output=True)
if result.stdout:
    print("⚠ Some processes still running, try again or use: kill -9 <PID>")
else:
    print("✓ All training processes terminated")

Stopping all training processes...
✓ Training processes stopped
✓ All training processes terminated
